In [1]:
import pandas as pd
import requests
import glob
import zipfile
import io
import os
import time

from io import BytesIO
import xml.etree.ElementTree as ET  # XML 파싱을 위해 추가
import logging  # 로깅을 위해 추가
from tqdm import tqdm

# 기업개황 개발가이드

In [53]:
# 공시대상회사의 고유번호(8자리)
corp_code = '01505760' # NAVER
url = f"https://opendart.fss.or.kr/api/company.json?crtfc_key={crtfc_key}&corp_code={corp_code}"

In [54]:
url

'https://opendart.fss.or.kr/api/company.json?crtfc_key=6cc25146da26f08fc798e31bc27f02eaf67d6cd2&corp_code=01505760'

In [30]:
# def search_rcepts(corp_code, crtfc_key, start_date, end_date):
    
#     # API 호출 및 응답 저장
#     url = f"https://opendart.fss.or.kr/api/list.json?crtfc_key={crtfc_key}&corp_code={corp_code}&bgn_de={start_date}&end_de={end_date}&page_no=1&page_count=10"

#     # API 호출
#     response = requests.get(url)

#     # 상태 코드 확인
#     if response.status_code == 200:
#         # JSON 형태로 변환
#         data = response.json()
#         reports = pd.DataFrame(data['list'])
#         return reports
        
#     else:
#         print(f"API 호출 실패: 상태 코드 {response.status_code}")
#         print(response.text)

        
# def download_and_extract_zip(url, extract_to='extracted_files'):
#     """
#     URL에서 ZIP 파일을 다운로드하고 지정된 디렉토리에 압축을 풉니다.
    
#     Args:
#         url (str): ZIP 파일을 다운로드할 URL
#         extract_to (str): 파일을 압축 해제할 디렉토리 (기본값: 'extracted_files')
#     """
#     print(f"ZIP 파일 다운로드 중: {url}")
    
#     # URL에서 파일 다운로드
#     response = requests.get(url)
    
#     # 응답 상태 확인
#     if response.status_code == 200:
#         # 압축 해제할 디렉토리 생성
#         os.makedirs(extract_to, exist_ok=True)
        
#         # 다운로드한 콘텐츠를 메모리 내 파일 객체로 변환
#         zip_file = io.BytesIO(response.content)
        
#         # ZIP 파일 압축 해제
#         with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#             # ZIP 파일 내용 출력
#             #print("ZIP 파일 내용:")
#             file_list = zip_ref.namelist()
#             for file in file_list:
#                 print(f"  {file}")
            
#             # 모든 파일 압축 해제
#             #print(f"압축 해제 중: {extract_to}")
#             zip_ref.extractall(extract_to)
        
#         #print(f"압축 해제 완료. 파일 개수: {len(file_list)}")
#         return file_list
#     else:
#         print(f"다운로드 실패: 상태 코드 {response.status_code}")
#         print(response.text)
#         return None

In [31]:
def search_rcepts(corp_code, crtfc_key, start_date, end_date):
    url = f"https://opendart.fss.or.kr/api/list.json?crtfc_key={crtfc_key}&corp_code={corp_code}&bgn_de={start_date}&end_de={end_date}&page_count=600"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        
        # 메시지 기반 예외 처리
        if data.get('status') != '000':
            print(f"[스킵] {corp_code}: {data.get('message')}")
            return pd.DataFrame()
        
        # list 키가 있을 경우만 데이터프레임 생성
        if 'list' in data:
            return pd.DataFrame(data['list'])
        else:
            print(f"[스킵] {corp_code}: list 없음")
            return pd.DataFrame()
    else:
        print(f"[에러] {corp_code}: status code {response.status_code}")
        return pd.DataFrame()

def download_and_extract_zip(url, extract_to='extracted_files'):
    """
    URL에서 ZIP 파일을 다운로드하고 지정된 디렉토리에 압축을 풉니다.
    
    Args:
        url (str): ZIP 파일을 다운로드할 URL
        extract_to (str): 파일을 압축 해제할 디렉토리 (기본값: 'extracted_files')
    """
    print(f"ZIP 파일 다운로드 중: {url}")
    
    # URL에서 파일 다운로드
    response = requests.get(url)
    
    # 응답 상태 확인
    if response.status_code == 200:
        # 압축 해제할 디렉토리 생성
        os.makedirs(extract_to, exist_ok=True)
        
        # 다운로드한 콘텐츠를 메모리 내 파일 객체로 변환
        zip_file = io.BytesIO(response.content)
        
        # ZIP 파일 압축 해제
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            # ZIP 파일 내용 출력
            #print("ZIP 파일 내용:")
            file_list = zip_ref.namelist()
            for file in file_list:
                print(f"  {file}")
            
            # 모든 파일 압축 해제
            #print(f"압축 해제 중: {extract_to}")
            zip_ref.extractall(extract_to)
        
        #print(f"압축 해제 완료. 파일 개수: {len(file_list)}")
        return file_list
    else:
        print(f"다운로드 실패: 상태 코드 {response.status_code}")
        print(response.text)
        return None

# DART API key and search period

In [32]:
crtfc_key = '6cc25146da26f08fc798e31bc27f02eaf67d6cd2'  # 기업통계팀 key로 변경
start_date = '20240101'
end_date = '20240530'   # 2025년 이후까지 늘려야?

TEST

In [52]:
r = search_rcepts('01505760', crtfc_key, start_date, end_date)
r['rcept_no'][0]
url = f"https://opendart.fss.or.kr/api/document.xml?crtfc_key={crtfc_key}&rcept_no={r['rcept_no'][0]}"
files = download_and_extract_zip(url, 'test')

[스킵] 01505760: 조회된 데이타가 없습니다.


KeyError: 'rcept_no'

In [34]:
r.count()

corp_code     100
corp_name     100
stock_code    100
corp_cls      100
report_nm     100
rcept_no      100
flr_nm        100
rcept_dt      100
rm            100
dtype: int64

# DART API: 공시검색
- corp_rcept_no: 각 기업코드별 공시보고서 접수번호(rcept_no) 매칭

In [35]:
# API 1첨 돌렸을 때 (01652527) 금강에스티피 에서 한도 초과 발생(총 8243개 법인만 호출 완료)

# start_time = time.time()  # 시작 시간 

# corp_rcept_no = pd.read_csv('corp_code.csv', dtype=str, encoding = 'cp949').set_index('corp_code')

# reports = pd.DataFrame()

# # 1000개 기업 공시 검색하는데 80초 소요
# for i in corp_rcept_no.index[:15000]:
#     r = search_rcepts(i, crtfc_key, start_date, end_date)
#     reports = pd.concat([reports, r], axis=0)

# end_time = time.time()  # 종료 시간

# print(f"⏱ 실행 시간: {end_time - start_time:.2f}초")

# reports.to_excel('reports_FY2023_1.xlsx', index=False)
# reports = pd.read_excel('reports_FY2023_1.xlsx')

In [50]:
# API 1첨 돌렸을 때 (01652527) 금강에스티피 에서 한도 초과 발생(총 8243개 법인만 호출 완료)

start_time = time.time()  # 시작 시간 

corp_rcept_no = pd.read_excel('corp_code.xlsx', dtype=str).set_index('corp_code')

reports = pd.DataFrame()

# 1000개 기업 공시 검색하는데 80초 소요
for i in corp_rcept_no.index[10135:10136]:
    r = search_rcepts(i, crtfc_key, start_date, end_date)
    reports = pd.concat([reports, r], axis=0)

end_time = time.time()  # 종료 시간

print(f"⏱ 실행 시간: {end_time - start_time:.2f}초")

⏱ 실행 시간: 1.45초


In [51]:
reports

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00857912,제이앤드,,E,호가중개시스템을통한소액매출공시서류,20240401000803,제이앤드,20240401,


In [64]:
reports.to_csv('nnaer.csv')

In [60]:
# API 1첨 돌렸을 때 (01652527) 금강에스티피 에서 한도 초과 발생(총 8243개 법인만 호출 완료)

start_time = time.time()  # 시작 시간 

corp_rcept_no = pd.read_excel('corp_code.xlsx', dtype=str).set_index('corp_code')

reports = pd.DataFrame()

# 1000개 기업 공시 검색하는데 80초 소요
for i in corp_rcept_no.index[:8243]:
    r = search_rcepts(i, crtfc_key, start_date, end_date)
    reports = pd.concat([reports, r], axis=0)

end_time = time.time()  # 종료 시간

print(f"⏱ 실행 시간: {end_time - start_time:.2f}초")

KeyboardInterrupt: 

In [15]:
reports

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00156691,한국공항,005430,Y,분기보고서 (2024.03),20240516001463,한국공항,20240516,
1,00156691,한국공항,005430,Y,연결재무제표기준영업(잠정)실적(공정공시),20240508800461,한국공항,20240508,유
2,00156691,한국공항,005430,Y,정기주주총회결과,20240321800717,한국공항,20240321,유
3,00156691,한국공항,005430,Y,사업보고서 (2023.12),20240313001250,한국공항,20240313,연
4,00156691,한국공항,005430,Y,감사보고서제출,20240306801051,한국공항,20240306,유
...,...,...,...,...,...,...,...,...,...
0,00200141,화성보일러,,E,감사보고서 (2023.12),20240401002147,솔공인회계사감사반(제14호),20240401,
0,01454208,하영에스엠씨,,E,감사보고서 (2023.12),20240401001285,정진회계법인,20240401,
0,01653270,에코클리어,,E,[기재정정]감사보고서 (2023.12),20240411000214,회계법인지평,20240411,
1,01653270,에코클리어,,E,감사보고서 (2023.12),20240402002706,회계법인지평,20240402,정


In [88]:
# reports.to_excel('reports_FY2023_1.xlsx', index=False)
reports = pd.read_excel('reports_FY2023_1.xlsx')

In [17]:
# reports = pd.read_excel('reports_FY2023_1.xlsx', dtype={'corp_code': str, 'rcept_no': str})

In [89]:
reports.groupby(['corp_cls']).count()

,corp_code,corp_name,stock_code,report_nm,rcept_no,flr_nm,rcept_dt,rm
corp_cls,,,,,,,,
E,12118,12118,704,12118,12118,12118,12118,4563
K,23686,23686,23686,23686,23686,23686,23686,14182
N,1080,1080,1080,1080,1080,1080,1080,688
Y,13995,13995,13995,13995,13995,13995,13995,9134


In [90]:
# ER = reports.loc[reports['corp_cls'].eq('E')]
# ER['corp_code'].nunique() # 311개 기업의 공시가 있음

In [91]:
ER = reports
ER['corp_code'].nunique()

8243

In [92]:
# ER_audit = ER.loc[ER['report_nm'].str.contains('사업|감사') & ~ER['report_nm'].str.contains('연결감사|첨부정정') & ER['report_nm'].str.contains('2023.12')]
# # 2023.12 기준 사업 또는 감사 보고서가 있는 기업(298개)
# ER_audit['corp_code'].nunique() 

In [93]:
ER_audit = ER.loc[
    ER['report_nm'].str.contains('사업|감사') &
    ~ER['report_nm'].str.contains('연결감사') &
    ~ER['report_nm'].str.contains('첨부정정') &
    ER['report_nm'].str.contains('2023.12', regex=False)
]
ER_audit['corp_code'].nunique() 

8242

In [94]:
ER_noaudit = ER.loc[~ER['corp_code'].isin(ER_audit['corp_code'].unique())]
# 2023.12 기준 사업 또는 감사 보고서가 없는 기업(13개)
ER_noaudit['corp_code'].nunique() # 2023.12 기준 사업 또는 감사 보고서가 없는 기업 1개

1

In [95]:
ER_audit['report_nm'].unique()

array(['사업보고서 (2023.12)', '감사보고서 (2023.12)', '[기재정정]사업보고서 (2023.12)',
       '[기재정정]감사보고서 (2023.12)', '[첨부추가]사업보고서 (2023.12)',
       '사업보고서제출기한연장신고서 (2023.12)'], dtype=object)

In [96]:
ER_audit.to_csv('test.csv',encoding='utf-8-sig')

PermissionError: [Errno 13] Permission denied: 'test.csv'

In [ ]:
# ER_audit_latest.to_csv('test.csv',encoding='utf-8-sig')

In [97]:
ER_audit_one = ER_audit.sort_values('rcept_no', ascending=False).drop_duplicates(subset='corp_code', keep='first')

# DART API: 공시서류원본파일 다운로드

In [28]:
# extract_dir = "ER_audit"  # 압축 해제할 디렉토리 지정

# # 전체 1000개 기업중 기타(E)기업이고, 2023.12 기준 감사 또는 사업보고서가 있는 기업 공시보고서 다운로드하고 압축해제
# for i in ER_audit['rcept_no']:
#     url = f"https://opendart.fss.or.kr/api/document.xml?crtfc_key={crtfc_key}&rcept_no={i}"
#     files = download_and_extract_zip(url, extract_dir)

In [29]:
def download_and_extract_zip(url, extract_to):
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                z.extractall(extract_to)
                return z.namelist()
        except zipfile.BadZipFile:
            raise ValueError("Not a valid ZIP file (possibly an error response)")
    else:
        raise ValueError(f"Download failed with status code {response.status_code}")

extract_dir = "ER_audit"
os.makedirs(extract_dir, exist_ok=True)

for rcept_no in tqdm(ER_audit_one['rcept_no'], desc="Downloading and extracting reports"):
    url = f"https://opendart.fss.or.kr/api/document.xml?crtfc_key={crtfc_key}&rcept_no={rcept_no}"
    try:
        files = download_and_extract_zip(url, extract_dir)
    except Exception as e:
        print(f"[SKIPPED] {rcept_no}: {e}")
        continue

In [98]:
ER_audit_one[ER_audit_one['corp_code'] == 1530030]

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
3098,1530030,양명,NaN,E,감사보고서 (2023.12),20240129000131,대주회계법인,20240129,NaN


In [99]:
ER_audit_one

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
45238,1011182,대영씨엔이,NaN,E,[기재정정]사업보고서 (2023.12),20240529000624,대영씨엔이,20240529,연
49417,137854,신안,NaN,E,[기재정정]감사보고서 (2023.12),20240527000238,태성회계법인,20240527,NaN
47948,1741645,어반씨앤디,NaN,E,감사보고서 (2023.12),20240527000021,회계법인길인,20240527,NaN
41948,1616941,에코프로씨엔지,NaN,E,[기재정정]감사보고서 (2023.12),20240523000523,삼정회계법인,20240524,NaN
37297,633835,HJ중공업,97230.0,Y,[기재정정]사업보고서 (2023.12),20240523000216,HJ중공업,20240523,연
...,...,...,...,...,...,...,...,...,...
3099,1371224,대운개발,NaN,E,감사보고서 (2023.12),20240207000766,우덕회계법인,20240207,NaN
1261,203740,대아탱크터미널,NaN,E,감사보고서 (2023.12),20240207000517,신승회계법인,20240207,NaN
2225,206190,피에이치지,NaN,E,감사보고서 (2023.12),20240206000482,삼화회계법인,20240206,NaN
3098,1530030,양명,NaN,E,감사보고서 (2023.12),20240129000131,대주회계법인,20240129,NaN


In [32]:
# import os
# import requests
# import zipfile
# import io
# from tqdm import tqdm

# def download_and_extract_zip(url, extract_to):
#     response = requests.get(url)
#     if response.status_code == 200:
#         try:
#             with zipfile.ZipFile(io.BytesIO(response.content)) as z:
#                 z.extractall(extract_to)
#                 return z.namelist()
#         except zipfile.BadZipFile:
#             raise ValueError("Not a valid ZIP file (possibly XML error message)")
#     else:
#         raise ValueError(f"HTTP error: {response.status_code}")

# # 압축 해제 디렉토리
# extract_dir = "ER_audit"
# os.makedirs(extract_dir, exist_ok=True)

# # 실패한 rcept_no 리스트 저장용
# skipped_list = []

# # 다운로드 및 압축 해제 루프
# for rcept_no in tqdm(ER_audit['rcept_no'], desc="Downloading and extracting reports"):
#     url = f"https://opendart.fss.or.kr/api/document.xml?crtfc_key={crtfc_key}&rcept_no={rcept_no}"
#     try:
#         files = download_and_extract_zip(url, extract_dir)
#     except Exception as e:
#         print(f"[SKIPPED] {rcept_no}: {e}")
#         skipped_list.append(rcept_no)
#         continue

# # 필요시 로그 저장
# if skipped_list:
#     with open("skipped_rcept_no.txt", "w") as f:
#         for item in skipped_list:
#             f.write(f"{item}\n")


# 접수번호 - 기업코드 - 보고서명 - 접수일자 - xml파일경로 매핑

In [100]:
xml_files = pd.DataFrame({'path':glob.glob('ER_audit/*.xml')})
xml_files['rcept_no'] = xml_files['path'].apply(lambda x: x.split('\\')[1][:14])

In [101]:
xml_files

,path,rcept_no
0,ER_audit\20240124000171_00760.xml,20240124000171
1,ER_audit\20240129000131_00760.xml,20240129000131
2,ER_audit\20240202000518_00760.xml,20240202000518
3,ER_audit\20240206000116_00760.xml,20240206000116
4,ER_audit\20240206000482_00760.xml,20240206000482
...,...,...
35726,ER_audit\20240531000983_00760.xml,20240531000983
35727,ER_audit\20240531000993_00760.xml,20240531000993
35728,ER_audit\20240531001854_00760.xml,20240531001854
35729,ER_audit\20240531002530_00760.xml,20240531002530


In [102]:
xml_files.head()

,path,rcept_no
0,ER_audit\20240124000171_00760.xml,20240124000171
1,ER_audit\20240129000131_00760.xml,20240129000131
2,ER_audit\20240202000518_00760.xml,20240202000518
3,ER_audit\20240206000116_00760.xml,20240206000116
4,ER_audit\20240206000482_00760.xml,20240206000482


In [103]:
ER_audit_one['rcept_no'] = ER_audit_one['rcept_no'].astype('str')

In [105]:
ER_xml = pd.merge(xml_files, ER_audit_one, on='rcept_no')
tmp_xml_count = ER_xml.groupby('rcept_no')['path'].count().to_frame('xml_count').reset_index()
ER_xml = pd.merge(ER_xml, tmp_xml_count, on='rcept_no')
ER_xml.to_csv('ER_xml_1.csv', index=False,encoding='utf-8-sig')

In [106]:
ER_xml[ER_xml['corp_code'] == 1530030]

,path,rcept_no,corp_code,corp_name,stock_code,corp_cls,report_nm,flr_nm,rcept_dt,rm,xml_count
1,ER_audit\20240129000131_00760.xml,20240129000131,1530030,양명,NaN,E,감사보고서 (2023.12),대주회계법인,20240129,NaN,1


In [107]:
tmp_xml_count = ER_xml.groupby('rcept_no')['path'].count().to_frame('xml_count').reset_index()

In [108]:
ER_xml.loc[ER_xml['xml_count'].ge(2)]

,path,rcept_no,corp_code,corp_name,stock_code,corp_cls,report_nm,flr_nm,rcept_dt,rm,xml_count
17,ER_audit\20240221001483.xml,20240221001483,134477,SNT다이내믹스,3570.0,Y,사업보고서 (2023.12),SNT다이내믹스,20240221,연,3
18,ER_audit\20240221001483_00760.xml,20240221001483,134477,SNT다이내믹스,3570.0,Y,사업보고서 (2023.12),SNT다이내믹스,20240221,연,3
19,ER_audit\20240221001483_00761.xml,20240221001483,134477,SNT다이내믹스,3570.0,Y,사업보고서 (2023.12),SNT다이내믹스,20240221,연,3
20,ER_audit\20240221001626.xml,20240221001626,398792,SNT모티브,64960.0,Y,사업보고서 (2023.12),SNT모티브,20240221,연,3
21,ER_audit\20240221001626_00760.xml,20240221001626,398792,SNT모티브,64960.0,Y,사업보고서 (2023.12),SNT모티브,20240221,연,3
...,...,...,...,...,...,...,...,...,...,...,...
11563,ER_audit\20240513000862.xml,20240513000862,119140,핸즈코퍼레이션,143210.0,Y,[기재정정]사업보고서 (2023.12),핸즈코퍼레이션,20240513,연,2
11564,ER_audit\20240513000862_00761.xml,20240513000862,119140,핸즈코퍼레이션,143210.0,Y,[기재정정]사업보고서 (2023.12),핸즈코퍼레이션,20240513,연,2
11581,ER_audit\20240529000624.xml,20240529000624,1011182,대영씨엔이,NaN,E,[기재정정]사업보고서 (2023.12),대영씨엔이,20240529,연,3
11582,ER_audit\20240529000624_00760.xml,20240529000624,1011182,대영씨엔이,NaN,E,[기재정정]사업보고서 (2023.12),대영씨엔이,20240529,연,3


In [109]:
ER_xml

,path,rcept_no,corp_code,corp_name,stock_code,corp_cls,report_nm,flr_nm,rcept_dt,rm,xml_count
0,ER_audit\20240124000171_00760.xml,20240124000171,1563135,파운드리더블유(구.로지스씨엘),NaN,E,감사보고서 (2023.12),대성삼경회계법인,20240124,NaN,1
1,ER_audit\20240129000131_00760.xml,20240129000131,1530030,양명,NaN,E,감사보고서 (2023.12),대주회계법인,20240129,NaN,1
2,ER_audit\20240206000482_00760.xml,20240206000482,206190,피에이치지,NaN,E,감사보고서 (2023.12),삼화회계법인,20240206,NaN,1
3,ER_audit\20240207000517_00760.xml,20240207000517,203740,대아탱크터미널,NaN,E,감사보고서 (2023.12),신승회계법인,20240207,NaN,1
4,ER_audit\20240207000766_00760.xml,20240207000766,1371224,대운개발,NaN,E,감사보고서 (2023.12),우덕회계법인,20240207,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
11579,ER_audit\20240527000021_00760.xml,20240527000021,1741645,어반씨앤디,NaN,E,감사보고서 (2023.12),회계법인길인,20240527,NaN,1
11580,ER_audit\20240527000238_00760.xml,20240527000238,137854,신안,NaN,E,[기재정정]감사보고서 (2023.12),태성회계법인,20240527,NaN,1
11581,ER_audit\20240529000624.xml,20240529000624,1011182,대영씨엔이,NaN,E,[기재정정]사업보고서 (2023.12),대영씨엔이,20240529,연,3
11582,ER_audit\20240529000624_00760.xml,20240529000624,1011182,대영씨엔이,NaN,E,[기재정정]사업보고서 (2023.12),대영씨엔이,20240529,연,3
